# Imputation

## import Library

In [102]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

In [103]:
m = [[1,np.nan,3],[np.nan,np.nan,6],[np.nan,9,3]]
m

[[1, nan, 3], [nan, nan, 6], [nan, 9, 3]]

In [104]:
imp_mean = SimpleImputer(missing_values= np.nan, strategy= 'mean')
imp_mean.fit(m)
df = [[np.nan, 1,np.nan],[4, 0, 6], [np.nan, 2, np.nan]]

In [105]:
p = imp_mean.transform(df)
# replace the missing data with the mean of orignal data of each columns not the current data 

# Holidays 

## Import Library

In [106]:
from workalendar.europe.united_kingdom import UnitedKingdom

In [107]:
UnitedKingdom().holidays()

[(datetime.date(2024, 1, 1), 'New year'),
 (datetime.date(2024, 3, 29), 'Good Friday'),
 (datetime.date(2024, 3, 31), 'Easter Sunday'),
 (datetime.date(2024, 4, 1), 'Easter Monday'),
 (datetime.date(2024, 5, 6), 'Early May Bank Holiday'),
 (datetime.date(2024, 5, 27), 'Spring Bank Holiday'),
 (datetime.date(2024, 8, 26), 'Late Summer Bank Holiday'),
 (datetime.date(2024, 12, 25), 'Christmas Day'),
 (datetime.date(2024, 12, 26), 'Boxing Day')]

In [108]:
from typing import List
from dateutil.relativedelta import relativedelta, TH
import datetime
from workalendar.usa import California

In [109]:
class holidays:
  def __init__(self, year):
    self.year = year
  
  
  def create_custom_holidays(self) -> List:
    self.new_holidays = datetime.datetime(self.year, 11,1) + relativedelta(weekday= TH(+4)) +datetime.timedelta(days = 1)
    self.custom_hlidays = California().holidays()
    self.custom_hlidays.append((self.new_holidays.date(),'Black Friday'))
    # return {k : v for (k, v) in custom_hlidays}
    return self.custom_hlidays
  
  def get_holidays_date(self) -> List:
    self.holidays_list = [u for (u, v) in self.create_custom_holidays()]
    return self.holidays_list
  
  def is_holiday(self, current_date: datetime.date):
    """Determine if we have a holiday"""
    if current_date in self.get_holidays_date():
      return True
    else :
      return False
  
  

In [110]:
today = datetime.date(2024, 1, 15)
ho = holidays(2024)
ho.create_custom_holidays()
ho.get_holidays_date()
ho.is_holiday(today)


True

# Date Annotation

In [111]:
import calendar
calendar.monthrange(2024,1)

(0, 31)

In [112]:
def year_anchor(current_date : datetime.date):
  return (
    (current_date - datetime.date(current_date.year, 1, 1)).days,
    (datetime.date(current_date.year, 12, 31) - current_date).days
  )

In [113]:
now = datetime.date.today()

In [114]:
year_anchor(now)

(79, 286)

In [115]:
calendar.monthcalendar(now.year, now.month)[2][calendar.FRIDAY]

15

In [116]:
def get_last_friday(current_date: datetime.date, weekday = calendar.FRIDAY):
  return max(week[weekday] for week in calendar.monthcalendar(current_date.year, current_date.month))

In [117]:
get_last_friday(now)

29

## Season

In [118]:
YEAR = 2021
seasons = [
  ('winter', (datetime.date(YEAR, 1, 1), datetime.date(YEAR, 3, 20))),
  ('spring', (datetime.date(YEAR, 3, 21), datetime.date(YEAR, 6, 20))),
  ('summer', (datetime.date(YEAR, 6, 21), datetime.date(YEAR, 9, 22))),
  ('autumn', (datetime.date(YEAR, 9, 23), datetime.date(YEAR, 12, 20))),
  ('winter', (datetime.date(YEAR, 12, 21), datetime.date(YEAR, 12, 31)))
]




def is_in_interval(current_date: datetime.date, seasons):
  # return next(season for (season , (start, end)) in seasons if start <= current_date.replace(year = YEAR) <= end) 
  
  
  
  
  # season_list = [season for (season , (start, end)) in seasons]
  for i,j in enumerate(seasons):
    if j[1][0] <= current_date.replace(year = YEAR) <= j[1][1] :
      return j[0]
    else:
      return False
  
    
    
    

In [119]:
now1 = datetime.date.today()

In [120]:
now1

datetime.date(2024, 3, 20)

In [121]:
is_in_interval(now1, seasons)

'winter'

# Automated Feature Extraction

In [122]:
import featuretools as ft
from featuretools.primitives import Minute, Hour, Day, Month, Year, Weekday

In [123]:
data = pd.DataFrame(
  {'Time':['2014-01-01 01:41:50',
           '2014-01-01 02:06:50',
           '2014-01-01 02:31:50',
           '2014-01-01 02:56:50',
           '2014-01-01 03:21:50'],
   
    'Target': [0, 0, 0, 0, 1]
  }
)
data

,Time,Target
0,2014-01-01 01:41:50,0
1,2014-01-01 02:06:50,0
2,2014-01-01 02:31:50,0
3,2014-01-01 02:56:50,0
4,2014-01-01 03:21:50,1


In [124]:
from datetime import datetime

In [125]:
data['index'] = data.index

In [126]:
# data.Time = data.Time.apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S'))

In [127]:
es = ft.EntitySet('My EntitySet')
es.add_dataframe(
  dataframe = data,
  dataframe_name = 'data_name', 
  index = 'index', 
  time_index = 'Time'
)
es

Entityset: My EntitySet
  DataFrames:
    data_name [Rows: 5, Columns: 3]
  Relationships:
    No relationships

In [130]:
fm, features = ft.dfs(
  entityset= es, 
  dataframes = data,
  target_dataframe_name = 'data_name',
  max_depth = 1,
  trans_primitives= [Minute, Hour, Day, Month, Year, Weekday])

In [131]:
fm

,Target,DAY(Time),HOUR(Time),MINUTE(Time),MONTH(Time),WEEKDAY(Time),YEAR(Time)
index,,,,,,,
0,0,1,1,41,1,2,2014
1,0,1,2,6,1,2,2014
2,0,1,2,31,1,2,2014
3,0,1,2,56,1,2,2014
4,1,1,3,21,1,2,2014
